<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [4]:
!pip install openai --quiet

In [9]:
get_ipython().system_raw(f"ollama pull {modelid}")

In [10]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [12]:
completion = llm.chat.completions.create(
    model=modelid,
    messages=[
        {"role": "system", "content": "Talk like a pirate."},
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        },
    ],
)

print(completion.choices[0].message.content)

You can use the ` isinstance()` function in Python to check if an object is an instance of a class.

Here's an example:

```python
class Animal:
    pass

class Dog(Animal):
    pass

my_dog = Dog()
print(isinstance(my_dog, Animal))  # Output: True
print(isinstance(my_dog, Dog))     # Output: True
print(isinstance(my_dog, str))     # Output: False
```

In this example, `isinstance()` checks if `my_dog` is an instance of either the `Animal` or `Dog` classes. It also returns `False` when checking for instances of other classes like `str`.

The general syntax is:

```python
class_name_obj = isinstance(obj, class)
```

Where:

*   `obj` is the object to be checked.
*   `class` is the class you want to check against.

Note that if you omit the second argument (the class), `isinstance()` checks if `obj` is an instance of a superclass or any subclass of the given class.
